In [55]:
# Libraries
import itertools                         # tools to generate Cartesian products (all combinations of A,C,G,T)
from collections import Counter          # efficient counter for k-mers
from Bio import SeqIO                    # FASTA file reading (Biopython)
import pandas as pd                      # table manipulation (DataFrame)
import matplotlib.pyplot as plt          # plotting figures
from matplotlib import cm                # color maps (colormaps)

# -------------------------------
# Parameters
# -------------------------------
arquivo_fasta = "HCoV NL63.fasta"       # path to the input FASTA file
k = 3                                     # k-mer length (fixed here as 3)
saida_excel = f"kmer_frequency_HCoV-NL63.xlsx"  # name of the output Excel file
saida_cgr_png = f"CGR_k{k}.png"           # name of the PNG file for the CGR image of the first sample

# -------------------------------
# Utility functions
# -------------------------------
def contar_kmers(sequence: str, k: int) -> Counter:
    """
    Counts all k-mers (only A,C,G,T) in a sequence.
    Ignores windows with ambiguous characters.
    """
    seq = sequence.upper()               # converts the sequence to uppercase for consistency
    n = len(seq)                         # sequence length
    counts = Counter()                   # initializes the k-mer counter
    for i in range(n - k + 1):           # slides a window of size k along the sequence
        kmer = seq[i:i+k]                # extracts the current k-mer
        if all(base in "ACGT" for base in kmer):  # validates that k-mer contains only A,C,G,T
            counts[kmer] += 1            # increments the count for a valid k-mer
    return counts                        # returns the Counter with counts

def montar_tabela_frequencias(fasta_path: str, k: int) -> pd.DataFrame:
    """
    Reads a FASTA file (one or more sequences) and builds a DataFrame
    in the format:
    Sequence_ID, AAA, AAC, ..., TTT, Total_Frequency
    """
    kmers = [''.join(p) for p in itertools.product('ACGT', repeat=k)]  # generates the ordered list of all possible k-mers
    linhas = []                                # list to accumulate final table rows

    for record in SeqIO.parse(fasta_path, "fasta"):  # iterates over each sequence in the FASTA file
        seq_id = record.description             # uses the full description as identifier (keeps long IDs)
        seq = str(record.seq).upper()           # gets the sequence as string and standardizes to uppercase

        counts = contar_kmers(seq, k)           # counts valid k-mers for this sequence
        total = sum(counts.values())            # total sum of valid k-mers (effective window count)

        linha = [counts.get(kmer, 0) for kmer in kmers]  # builds a row with counts for each k-mer in order
        linha.append(total)                     # adds the total count at the end
        linhas.append([seq_id] + linha)         # aggregates: Sequence_ID + counts + Total_Frequency

    colunas = ["Sequence_ID"] + kmers + ["Total_Frequency"]  # column names in same order as row
    df = pd.DataFrame(linhas, columns=colunas)  # creates DataFrame with all samples
    return df                                   # returns the completed table

def probs_de_counts(counts: Counter) -> dict:
    """Converts a k-mer count Counter into probabilities (sum = 1)."""
    total = sum(counts.values())               # total k-mer occurrences
    if total == 0:                             # handles cases of short/invalid sequences
        return {}                              # no possible probabilities
    return {k: v / total for k, v in counts.items()}  # normalizes each count by total

def chaos_game_representation(probabilidades: dict, k: int):
    """
    Generates the CGR matrix (2^k x 2^k) for k-mer probabilities.
    Mapping:
      A -> bottom-left corner
      C -> top-left corner
      G -> bottom-right corner
      T -> top-right corner
    """
    size = 2 ** k                             # matrix dimension (number of subdivisions per axis)
    chaos = [[0.0] * size for _ in range(size)]  # initializes 2D matrix with zeros (probability values)

    for kmer, value in probabilidades.items():  # iterates over each k-mer and its probability
        maxx = maxy = size                    # current subquadrant limits in x and y
        posx = posy = 0.0                     # origin of current subquadrant
        for char in kmer:                     # traverses k-mer letters to recursively descend into subquadrants
            halfx, halfy = maxx / 2.0, maxy / 2.0  # half of the current quadrant size
            if char == "G":
                posx += halfx                 # G: move to right half (initially bottom-right)
            elif char == "C":
                posy += halfy                 # C: move to upper half (top-left)
            elif char == "T":
                posx += halfx                 # T: move right
                posy += halfy                 # and up (top-right)
            # for 'A', stay in bottom-left quadrant (no offset)
            maxx, maxy = halfx, halfy         # shrink current quadrant and continue refinement
        ix, iy = int(posx), int(posy)         # convert continuous positions to integer indices
        ix = min(ix, size - 1)                # safety: limit to right edge
        iy = min(iy, size - 1)                # safety: limit to top edge
        chaos[iy][ix] = value                 # assign the k-mer probability to the final cell
    return chaos                               # return CGR matrix

def plotar_cgr(chaos_matrix, titulo: str, nome_png: str):
    """Plots and saves the CGR image."""
    plt.figure(figsize=(5, 5))                # creates a square figure
    plt.title(titulo)                         # adds title
    plt.imshow(chaos_matrix, interpolation="nearest", cmap=cm.gray_r)  # renders matrix in inverted grayscale
    plt.axis('off')                           # hides axes for better visualization
    plt.tight_layout()                        # automatically adjusts margins
    plt.savefig(nome_png, dpi=300)            # saves image with high resolution
    plt.close()                               # closes figure to free memory

# -------------------------------
# Execution
# -------------------------------
# 1) Builds the complete table (all sequences, all k-mers, Total_Frequency)
df = montar_tabela_frequencias(arquivo_fasta, k)  # creates DataFrame with k-mer counts and totals per sample
df.to_excel(saida_excel, index=False)              # exports table to Excel (one sheet, no index)
print(f"Table generated: {saida_excel}")           # prints path to created file

# 2) Generates CGR image for the first sample only
#    (uses k-mer probabilities from that first sequence)
primeiro_record = next(SeqIO.parse(arquivo_fasta, "fasta"), None)  # gets first sequence from FASTA (or None)
if primeiro_record is not None:                 # checks if at least one sequence exists
    # AJUSTE: filtra para manter apenas A,C,G,T da 1ª amostra
    seq_primeira = "".join(base for base in str(primeiro_record.seq).upper() if base in "ACGT")
    counts_primeira = contar_kmers(seq_primeira, k)  # counts k-mers for the first sample
    probs_primeira = probs_de_counts(counts_primeira)  # converts counts to probabilities

    chaos = chaos_game_representation(probs_primeira, k)  # builds CGR matrix from probabilities
    titulo = f"CGR – {primeiro_record.id} – k={k}"        # prepares informative title
    plotar_cgr(chaos, titulo, saida_cgr_png)              # plots and saves CGR image
    print(f"CGR image of the 1st sample saved as: {saida_cgr_png}")  # success message
else:
    print("No sequence found in FASTA.")                  # message if FASTA is empty


Table generated: kmer_frequency_HCoV-NL63.xlsx
CGR image of the 1st sample saved as: CGR_k3.png
